# import

**ffht**
- [Python 2.7](https://github.com/FALCONN-LIB/FFHT)
- [Python 3.x](https://github.com/ahazxm/FFHT)
```sh
git clone https://github.com/ahazxm/FFHT.git
cd FFHT
python setup.py install
python example.py
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.stats import cauchy
from scipy.spatial import distance
import scipy.stats as stats

from sklearn import datasets
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import euclidean_distances

import sys
import ffht
from hashlib import md5
import timeit
import math

mbyte = 1048576
norm_l2 = Normalizer(norm='l2')
norm_l1 = Normalizer(norm='l1')

# tools

In [ ]:
from hashlib import md5

def hybrid_dist(x, y, d_max, verbose=False):
    """
    Args:
        x1, y1: hamming 距离向量
        x2, y2: 欧式距离向量
        verbose: 显示详细内容
    """
    hd = distance.hamming(x[0], y[0])
    ed = distance.euclidean(x[1], y[1]) / d_max
    hybrid_d = 0.5 * hd + 0.5 * ed
    
    if verbose is True:
        print('hamming dist = {:.4f}, euclidean dist = {:.4f}'.format(hd, ed))
        print('hybrid dist = {}'.format(hybrid_d))
        
    return hybrid_d

def gram_schmidt(A):
    """
    Desc:
        Gram-schmidt正交化
        将所矩阵正交化
    """
    Q=np.zeros_like(A)
    cnt = 0
    for a in A.T:
        u = np.copy(a)
        for i in range(0, cnt):
            u -= np.dot(np.dot(Q[:, i].T, a), Q[:, i]) # 减去待求向量在以求向量上的投影
        e = u / np.linalg.norm(u)  # 归一化
        Q[:, cnt] = e
        cnt += 1
    R = np.dot(Q.T, A)
    return Q

def generate_md5(H):
    """
    Desc:
        md5 编码
    """
    hmd5 = md5()
    hmd5.update(str(H).encode(encoding='utf-8'))
    return hmd5.hexdigest()

def transform_hybrid(P1, P2, d):
    return np.append(P1, P2).reshape(2, -1).T.reshape(-1, 2, d)

def feature_hashing(P, target_d, random_state=None):
    """
    Desc:
        特征哈希
    Args:
        P: 点
        target_d: 降维度  
    """

    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState()

    d = P.shape[1]

    zero_idxs = rand.randint(0, target_d, d)
    signs = rand.choice([-1, 1], d)
    S = np.zeros((target_d, d))
    for i, idx in enumerate(zero_idxs):
        S[idx][i] = signs[i]
    
    return np.array([S.dot(p) for p in P])

# 实验指标

1. 查询到最近邻的概率
$$
pr = \frac{N_{find}}{N}
$$
2. 准确率(现更改为错误率 error_ratio)
$$
E = \frac{1}{k}\sum^k_{i=1}\frac{||q-N(q)_i||}{||q-I(q)_i||}
$$
    - $N(q)_i$ 真实的$i$近邻
    - $I(q)_i$ 查询的$i$近邻
3. 召回率
4. 候选集数量(现改成 selectivity, 候选数量/数据集大小)
5. 哈希时间
6. 查询时间(最近邻)
7. 空间消耗由于不同方法存储结构不同，放至函数体外

In [ ]:
def metrics(P, query_func, args, test, dataset_title, exact_nearest_neighbor, exact_k_near_neighbors, verbose=False):
    """
    Desc:
        评估
            1. 查询到最近邻的概率
            2. 准确率(现更改为错误率 error_ratio)
            3. 召回率
            4. 候选集数量(现改成 selectivity, 候选数量/数据集大小)
            5. 哈希时间
            6. 查询时间(最近邻)
            7. 空间消耗由于不同方法存储结构不同，放至函数体外
    Args:
        P: 数据集
        query_func: 查询函数
        args: 查询函数所需参数
        test: 测试集
        dataset_title: 
        exact_nearest_neighbor: 精确的最近邻
        exact_k_near_neighbors: 精确的k-近邻
    """
    n = P.shape[0]
    K = 50 # k近邻
    correct = 0
    hash_times = 0.0
    total_times = 0.0
    n_test = 0
    recall = 0.0
    error_ratio = 0.0
    selectivity = 0.0
    
    for i, q in enumerate(test):
        
        t1 = timeit.default_timer()
        candidates = query_func(args, q)
        t2 = timeit.default_timer()
        
        if len(candidates) < K: # 排除异常点
            continue
        
        hash_times += (t2 - t1 + 0.0) # 5.
        selectivity += len(candidates) / n # 4.
        
        t1 = timeit.default_timer()
        candidates_dist = np.array([hybrid_dist(q, P[idx], dmax[dataset_title]) for idx in candidates])
        nearest_neighbor_idx = candidates[np.argsort(candidates_dist)[0]]        
        t2 = timeit.default_timer()
        
        total_times += (t2 - t1 + 0.0) # 6.
        
        if nearest_neighbor_idx == exact_nearest_neighbor[i]:
            correct += 1

        k_near_neighbors = candidates[np.argsort(candidates_dist)[:K]]
        n_correct_k = len(np.intersect1d(exact_k_near_neighbors[i], k_near_neighbors))
        
        recall += n_correct_k / K # 3 召回率
        error_ratio += np.array([hybrid_dist(q, P[exact_k_near_neighbors[i][idx]], dmax[dataset_title]) / 
                 hybrid_dist(q, P[k_near_neighbors[idx]], dmax[dataset_title]) for idx in range(K)]).sum() / K # 2. 错误率        
        
        n_test += 1 # 有效测试点数量
        
    pr = correct / n_test # 1
    error_ratio = error_ratio / n_test # 2
    recall = recall / n_test # 3
    selectivity = selectivity / n_test # 4
    hash_times = hash_times / n_test # 5
    total_times = (total_times + hash_times) / n_test # 6
    
    if verbose is True:
        print('The probability of find the nearest neighbor is {:.4f}'.format(pr))
        print('Error ratio: {:.4f}, Recall: {:.4f}, Selectivity: {:.4f}'.format(error_ratio, 
                                                                                recall, 
                                                                                selectivity))
        print('Hash time: {:.4f}, total time: {:.4f}'.format(hash_times, total_times))

    return pr, error_ratio, recall, selectivity, hash_times, total_times

# Datasets

[synset](https://github.com/7thMar/synset) **synset-1e4d128-train/test.csv**

In [ ]:
K = 50

In [ ]:
dic_nearest_neighbor = dict()
dic_k_near_neighbors = dict()
dmax = dict()

In [ ]:
train = np.loadtxt('../synset/OPRCP/synset-1e4d128-train.csv').reshape([-1, 2, 128])
test = np.loadtxt('../synset/OPRCP/synset-1e4d128-test.csv').reshape([-1, 2, 128])
dataset_title = '1e4d128'

In [ ]:
dmax[dataset_title] = euclidean_distances(train[:, 1, :]).max()

存储 **最近邻** 与 **K近邻**

In [ ]:
%%time
nearest_neighbor = []
k_near_neighbors = []
for q in test[:100]:
    ds = np.array([hybrid_dist(q, p, dmax[dataset_title]) for p in train])
    order_ds_args = np.argsort(ds)
    nearest_neighbor.append(order_ds_args[0])
    k_near_neighbors.append(order_ds_args[:K])
    
dic_nearest_neighbor[dataset_title] = nearest_neighbor
dic_k_near_neighbors[dataset_title] = k_near_neighbors

# 对比函数

## OPRCP

> 标准 OPR，CP

In [ ]:
def generate_cross_polytope(d, random_state=None):
    """
    Desc:
        生成正轴型 d 维, 2 * d 个
    """
    if random_state is not None:
        np.random.seed(random_state)
    A = np.random.normal(0, 1, (d, d))
    A = norm_l2.fit_transform(gram_schmidt(A))
    A = np.append(A, -A).reshape(-1, d)
    return A

#### 参数类

In [ ]:
def OPRCP(P, k, L, random_state=None):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
        center: 点集均值
        e： 正轴型坐标
        As: 旋转向量
        seeds: 排列种子（用于进行相同的打乱重排过程）
        auxiliary_vector: 辅助向量
        buckets: 桶
    """
    n = P.shape[0]
    d = P.shape[2]
    sd = int(d / k)
    C = math.ceil(d / k + L) + 2
    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    print('k = {}, L = {}'.format(k, L))
    print('Every segament length is {}'.format(sd))

    buckets = []

    seeds = np.random.choice(np.arange(k * L * d), L, replace=False)
    one_segament_vector = list(range(sd, 0, -1))
    auxiliary_vector = np.array(one_segament_vector * k)
    e = generate_cross_polytope(d, random_state) # 生成正轴型向量 2 * d 个
    As = np.array([norm_l2.fit_transform(A) for A in np.random.normal(0, 1, (L, d, d))]) # 生成辅助旋转矩阵

    for i in range(L):
        bucket = dict()
        A = As[i]

        # 旋转
        rotate = norm_l2.fit_transform([A.dot(p) for p in P2])
        # 取离的最近的点的索引
        H2 = rotate.dot(e.T).argmax(axis=1)

        for idx, p in enumerate(P):
            np.random.seed(seeds[i])
            permutate_p = np.random.permutation(p[0])

            H1 = -((auxiliary_vector * permutate_p).reshape(-1, sd).max(axis=1) - sd)
            H1[H1 == sd] = -1

            for j, h in enumerate(H1):
                if h == -1:
                    nj = j
                    while H1[nj] == -1:
                        nj = (1 + nj) % k
                    H1[j] = H1[nj] + C

            H = np.append(H1, H2[idx]) # H=(H1, H2)
            bi = generate_md5(H)
            if bi not in bucket:
                bucket[bi] = [idx]
            else:
                bucket[bi].append(idx)

        buckets.append(bucket)

    return center, e, As, seeds, auxiliary_vector, buckets

单层存储空间缩小

#### 查询函数

In [ ]:
def OPRCP_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
    Returns:
    
    """
    center, e, As, seeds, auxiliary_vector, buckets = args[0]
    m = args[1]
    k, L = args[2]
    
    q1 = q[0].copy()
    d = q1.shape[0]
    
    C = math.ceil(d / k + L) + 2
    sd = int(d / k)

    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []

    for i in range(L):
        bucket = dict()

        np.random.seed(seeds[i])
        q1_ = np.random.permutation(q1)
        H1 = -((auxiliary_vector * q1_).reshape(-1, sd).max(axis=1) - sd)
        H1[H1 == sd] = -1

        for j, h in enumerate(H1):
            if h == -1:
                nj = j
                while H1[nj] == -1:
                    nj = (1 + nj) % k
                H1[j] = H1[nj] + C
        
        rotate = norm_l2.fit_transform([As[i].dot(q2)])[0]
        H2 = rotate.dot(e.T).argsort()[::-1][:m]
        
        for h2 in H2:
            H = np.append(H1, h2)
            bi = generate_md5(H)
            if bi in buckets[i]:
                result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 4
L = 34
m = 2
oprcp = OPRCP(train, k=k, L=L)

In [ ]:
m = 2

In [ ]:
center, e, As, seeds, auxiliary_vector, buckets = oprcp
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, OPRCP_query, [oprcp, m, (k, L)],
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## OPRCP version 2

1. 加速旋转过程（库函数替代循环过程）
2. 构建阶段进行多探测（即生成的哈希值对应最近的 **m** 个索引值）
3. 查询阶段不使用多探测

### 处理函数

In [ ]:
def OPRCP2(P, k, L, m, random_state=None):
    n = P.shape[0]
    d = P.shape[2]

    sd = int(d / k)
    C = math.ceil(d / k + L) + 2

    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()

    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    buckets = []

    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState()
    
    # OPR param
    seeds = rand.choice(np.arange(k * L * d), L, replace=False)
    one_segament_vector = list(range(sd, 0, -1))
    auxiliary_vector = np.array(one_segament_vector * k)

    # CP param
    e = generate_cross_polytope(d, random_state)[:d] # 生成正轴型向量 2 * d 个
    As = np.array([norm_l2.fit_transform(A) for A in np.random.normal(0, 1, (L, d, d))]) # 生成辅助旋转矩阵
    
    for i in range(L):
        bucket = dict()
        A = As[i]
        
        # 库函数替代循环从而加速旋转过程
        rotate = norm_l2.fit_transform(A.dot(P2.T).T)
        # 对距离进行排序（对应最优方法，未完全实现）
        H2 = np.abs(rotate.dot(e.T)).argsort(axis=1)
        for idx, p in enumerate(P):
            np.random.seed(seeds[i])
            permutate_p = np.random.permutation(p[0])

            H1 = -((auxiliary_vector * permutate_p).reshape(-1, sd).max(axis=1) - sd)
            H1[H1 == sd] = -1

            for j, h in enumerate(H1):
                if h == -1:
                    nj = j
                    while H1[nj] == -1:
                        nj = (1 + nj) % k
                    H1[j] = H1[nj] + C

            # 构建阶段进行多探测，取前m近的索引为哈希值
            for h2 in H2[idx][::-1][:m]:
                bi = generate_md5([H1, h2])
                if bi not in bucket:
                    bucket[bi] = [idx]
                else:
                    bucket[bi].append(idx)

        buckets.append(bucket)

    return center, e, As, seeds, auxiliary_vector, buckets

### 查询函数

In [ ]:
def OPRCP2_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
    """
    center, e, As, seeds, auxiliary_vector, buckets = args[0]
    m, k, L = args[1]


    q1 = q[0].copy()
    d = q1.shape[0]
    C = math.ceil(d / k + L) + 2
    sd = int(d / k)

    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []

    for i in range(L):
        np.random.seed(seeds[i])
        permutate_q = np.random.permutation(q1)
        H1 = -((auxiliary_vector * permutate_q).reshape(-1, sd).max(axis=1) - sd)
        H1[H1 == sd] = -1

        for j, h in enumerate(H1):
            if h == -1:
                nj = j
                while H1[nj] == -1:
                    nj = (1 + nj) % k
                H1[j] = H1[nj] + C

        rotate = norm_l2.fit_transform([As[i].dot(q2)])[0]
        H2 = np.abs(rotate.dot(e.T)).argmax()
        bi = generate_md5([H1, H2])
        if bi in buckets[i]:
            result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 4
L = 20
m = 3
oprcp = OPRCP2(train, k, L, m, random_state=1)

In [ ]:
size = 0
for bucket in oprcp[-1]:
    for key, b in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(b)
size / mbyte

In [ ]:
%%time
metrics(train, OPRCP2_query, [oprcp, (m, k, L)],
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## OPRCP version 3

1. **fast hadamard** 实现伪旋转替代真旋转（加速旋转过程）
2. 构建阶段进行多探测（即生成的哈希值对应最近的 **m** 个索引值）
3. 查询阶段不使用多探测

### 处理

In [ ]:
def generate_cross_polytope(d, random_state=None):
    """
    Desc:
        生成正轴型 d 维, 2 * d 个
    """
    if random_state is not None:
        np.random.seed(random_state)
    A = np.random.normal(0, 1, (d, d))
    A = norm_l2.fit_transform(gram_schmidt(A))
    A = np.append(A, -A).reshape(-1, d)
    return A

In [ ]:
def OPRCP3(P, k, L, m, num_rotations=3, random_state=None):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
        
    """
    n = P.shape[0]
    d = P.shape[2]
    sd = int(d / k)
    C = math.ceil(d / k + L) + 2
    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    print('k = {}, L = {}'.format(k, L))
    print('Every segament length is {}'.format(sd))

    buckets = []
    
    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState(random_state)
    
    seeds = rand.choice(np.arange(k * L * d), L, replace=False)
    one_segament_vector = list(range(sd, 0, -1))
    auxiliary_vector = np.array(one_segament_vector * k)
    e = generate_cross_polytope(d, random_state) # 生成正轴型向量 2 * d 个
    e = e[:d]
    Ds = rand.choice([-1, 1], (L, num_rotations, d))

    idxs = np.arange(n)

    
    for i in range(L):
        bucket = dict()
        rotate = []
        
        # fast hadamard 实现伪旋转替代真旋转
        for p in P2:
            for j in range(num_rotations):
                p = p * Ds[i][j]
                ffht.fht(p)
            rotate.append(p)
        rotate = np.array(rotate)
        rotate = norm_l2.fit_transform(rotate)
        H2 = np.abs(rotate.dot(e.T)).argsort(axis=1)

        for idx, p in enumerate(P):
            np.random.seed(seeds[i])
            php = np.random.permutation(p[0])

            H1 = -((auxiliary_vector * php).reshape(-1, sd).max(axis=1) - sd)
            H1[H1 == sd] = -1

            for j, h in enumerate(H1):
                if h == -1:
                    nj = j
                    while H1[nj] == -1:
                        nj = (1 + nj) % k
                    H1[j] = H1[nj] + C
           
            for h2 in H2[idx][::-1][:m]:
                H = np.append(H1, h2)

                bi = generate_md5(H)
                if bi not in bucket:
                    bucket[bi] = [idx]
                else:
                    bucket[bi].append(idx)

        buckets.append(bucket)

    return center, e, Ds, seeds, auxiliary_vector, buckets

### 查询

In [ ]:
def OPRCP3_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
    """
    center, e, Ds, seeds, auxiliary_vector, buckets = args[0]
    m = args[1]
    k, L = args[2]
    num_rotations = args[3]

    q1 = q[0].copy()
    d = q1.shape[0]
    C = math.ceil(d / k + L) + 2
    
    sd = int(d / k)
    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []

    for i in range(L):
        np.random.seed(seeds[i])
        q1_ = np.random.permutation(q1)
        H1 = -((auxiliary_vector * q1_).reshape(-1, sd).max(axis=1) - sd)
        H1[H1 == sd] = -1

        for j, h in enumerate(H1):
            if h == -1:
                nj = j
                while H1[nj] == -1:
                    nj = (1 + nj) % k
                H1[j] = H1[nj] + C

        q3 = q2.copy()
        for j in range(num_rotations):
            q3 = q3 * Ds[i][j]
            ffht.fht(q3)
        rotate = norm_l2.fit_transform([q3])[0]
        H2 = np.abs(rotate.dot(e.T)).argmax()
        H = np.append(H1, H2)
        bi = generate_md5(H)
        if bi in buckets[i]:
            result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 2
L = 10
m = 3
num_rotations = 3
oprcp = OPRCP3(train, k=k, L=L, m=m, num_rotations=num_rotations)

In [ ]:
center, e, Ds, seeds, auxiliary_vector, buckets = oprcp
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, OPRCP3_query, [oprcp, m, (k, L), num_rotations],
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## OPRCP version 4

1. **fast hadamard** 实现伪旋转替代真旋转（加速旋转过程）

### 处理

In [ ]:
def OPRCP4(P, k, L, num_rotations=3, random_state=None):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
        
    """
    n = P.shape[0]
    d = P.shape[2]
    sd = int(d / k)
    C = math.ceil(d / k + L) + 2
    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    print('k = {}, L = {}'.format(k, L))
    print('Every segament length is {}'.format(sd))

    buckets = []
    
    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState(random_state)
    
    seeds = rand.choice(np.arange(k * L * d), L, replace=False)
    one_segament_vector = list(range(sd, 0, -1))
    auxiliary_vector = np.array(one_segament_vector * k)
    e = generate_cross_polytope(d, random_state) # 生成正轴型向量 2 * d 个
    Ds = rand.choice([-1, 1], (L, num_rotations, d))


    for i in range(L):
        bucket = dict()

        rotate = []
        for p in P2:
            for j in range(num_rotations):
                p = p * Ds[i][j]
                ffht.fht(p)
            rotate.append(p)
        rotate = np.array(rotate)
        rotate = norm_l2.fit_transform(rotate) # 加速
        H2 = np.abs(rotate.dot(e[:d].T)).argmax(axis=1)

        for idx, p in enumerate(P):

            np.random.seed(seeds[i])
            php = np.random.permutation(p[0])

            H1 = -((auxiliary_vector * php).reshape(-1, sd).max(axis=1) - sd)
            H1[H1 == sd] = -1

            for j, h in enumerate(H1):
                if h == -1:
                    nj = j
                    while H1[nj] == -1:
                        nj = (1 + nj) % k
                    H1[j] = H1[nj] + C

            H = np.append(H1, H2[idx])
            # print(H)
            bi = generate_md5(H)
            if bi not in bucket:
                bucket[bi] = [idx]
            else:
                bucket[bi].append(idx)
       #  print(i)
        buckets.append(bucket)

    return center, e, Ds, seeds, auxiliary_vector, buckets

### 查询

In [ ]:
def OPRCP4_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
    """
    center, e, Ds, seeds, auxiliary_vector, buckets = args[0]
    m = args[1]
    k, L = args[2]
    num_rotations = args[3]
    
    q1 = q[0].copy()
    d = q1.shape[0]
    C = math.ceil(d / k + L) + 2
    sd = int(d / k)
    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []

    for i in range(L):
        bucket = dict()

        np.random.seed(seeds[i])
        q1_ = np.random.permutation(q1)
        H1 = -((auxiliary_vector * q1_).reshape(-1, sd).max(axis=1) - sd)
        H1[H1 == sd] = -1

        for j, h in enumerate(H1):
            if h == -1:
                nj = j
                while H1[nj] == -1:
                    nj = (1 + nj) % k
                H1[j] = H1[nj] + C
        
        q3 = q2.copy()
        for j in range(num_rotations):
            q3 = q3 * Ds[i][j]
            ffht.fht(q3)
        rotate = norm_l2.fit_transform([q3])[0]
        H2 = np.abs(rotate.dot(e[:d].T)).argsort()[::-1][:m]
        
        for h2 in H2:
            H = np.append(H1, h2)
            bi = generate_md5(H)
            if bi in buckets[i]:
                result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 2
L = 10
num_rotations = 3
oprcp = OPRCP4(train, k=k, L=L, num_rotations=num_rotations)

In [ ]:
m = 2

In [ ]:
center, e, Ds, seeds, auxiliary_vector, buckets = oprcp
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, OPRCP4_query, [oprcp, m, (k, L), num_rotations],
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## OPRCP version 5

1. 加速旋转过程（库函数替代循环过程）
2. 构建阶段进行多探测（即生成的哈希值对应最近的 **m** 个索引值）
3. 查询阶段使用多探测

### 处理函数

In [ ]:
def OPRCP5(P, k, L, m, random_state=None):
    n = P.shape[0]
    d = P.shape[2]

    sd = int(d / k)
    C = math.ceil(d / k + L) + 2

    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()

    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    buckets = []

    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState()
    
    # OPR param
    seeds = rand.choice(np.arange(k * L * d), L, replace=False)
    one_segament_vector = list(range(sd, 0, -1))
    auxiliary_vector = np.array(one_segament_vector * k)

    # CP param
    e = generate_cross_polytope(d, random_state)[:d] # 生成正轴型向量 2 * d 个
    As = np.array([norm_l2.fit_transform(A) for A in np.random.normal(0, 1, (L, d, d))]) # 生成辅助旋转矩阵
    
    for i in range(L):
        bucket = dict()
        A = As[i]
        
        # 库函数替代循环从而加速旋转过程
        rotate = norm_l2.fit_transform(A.dot(P2.T).T)
        # 对距离进行排序（对应最优方法，未完全实现）
        H2 = np.abs(rotate.dot(e.T)).argsort(axis=1)
        for idx, p in enumerate(P):
            np.random.seed(seeds[i])
            permutate_p = np.random.permutation(p[0])

            H1 = -((auxiliary_vector * permutate_p).reshape(-1, sd).max(axis=1) - sd)
            H1[H1 == sd] = -1

            for j, h in enumerate(H1):
                if h == -1:
                    nj = j
                    while H1[nj] == -1:
                        nj = (1 + nj) % k
                    H1[j] = H1[nj] + C

            # 构建阶段进行多探测，取前m近的索引为哈希值
            for h2 in H2[idx][::-1][:m]:
                bi = generate_md5([H1, h2])
                if bi not in bucket:
                    bucket[bi] = [idx]
                else:
                    bucket[bi].append(idx)

        buckets.append(bucket)

    return center, e, As, seeds, auxiliary_vector, buckets

### 查询函数

In [ ]:
def OPRCP5_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
    """
    center, e, As, seeds, auxiliary_vector, buckets = args[0]
    m, k, L = args[1]


    q1 = q[0].copy()
    d = q1.shape[0]
    C = math.ceil(d / k + L) + 2
    sd = int(d / k)

    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []

    for i in range(L):
        np.random.seed(seeds[i])
        permutate_q = np.random.permutation(q1)
        H1 = -((auxiliary_vector * permutate_q).reshape(-1, sd).max(axis=1) - sd)
        H1[H1 == sd] = -1

        for j, h in enumerate(H1):
            if h == -1:
                nj = j
                while H1[nj] == -1:
                    nj = (1 + nj) % k
                H1[j] = H1[nj] + C

        rotate = norm_l2.fit_transform([As[i].dot(q2)])[0]

    
        # 查询阶段同样使用多探测
        H2 = np.abs(rotate.dot(e.T)).argsort()[::-1][:m]
        for h in H2:
            bi = generate_md5([H1, h])
            if bi in buckets[i]:
                result.append(buckets[i][bi])
    

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 4
L = 12
m = 3
oprcp = OPRCP5(train, k, L, m, random_state=1)

In [ ]:
size = 0
for bucket in oprcp[-1]:
    for key, b in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(b)
size / mbyte

In [ ]:
%%time
metrics(train, OPRCP5_query, [oprcp, (m, k, L)],
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## MINCP version 1

### 处理

In [ ]:
def MINCP_version1(P, k, L, num_rotations=3, random_state=None):
    """
    Desc:
        Min-hash + cross polytope
    Args:
        P: 点集 n * 2 * d
        k: 
        L: 桶数
    Returns:
        center: mean
        e: cross-polytope
        Ds: fast hadamard 
        H_mat: min hash
        buckets
    """
    n = P.shape[0]
    d = P.shape[2]

    P1 = P[
        :, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    buckets = []
    for i in range(L):
        buckets.append(dict())
    
    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState(random_state)
    
    num_hash = k * L

    # 1. 生成 hash 函数
    h_mat = np.arange(1, d + 1)
    H_mat = np.array([rand.permutation(h_mat) for _ in range(num_hash)])

    e = generate_cross_polytope(d, random_state) # 生成正轴型向量 2 * d 个
    e = e[:d]
    Ds = rand.choice([-1, 1], (L, num_rotations, d))

    H2 = []
    for i in range(L):
        rotate = []
        for p in P2:
            for j in range(num_rotations):
                p = p * Ds[i][j]
                ffht.fht(p)
            rotate.append(p)
        rotate = norm_l2.fit_transform(np.array(rotate)) # 加速
        H2.append(np.abs(rotate.dot(e.T)).argmax(axis=1))

    for idx, p in enumerate(P1):
        H1 = (p * H_mat).astype(np.float)
        H1[H1 == 0] = np.inf
        H1 = H1.min(axis=1).reshape(L, k)
        for i in range(L):
            H = np.append(H1[i], H2[i][idx])
            bi = generate_md5(H)
            if bi not in buckets[i]:
                buckets[i][bi] = [idx]
            else:
                buckets[i][bi].append(idx)

    return center, e, Ds, H_mat, buckets

### 查询

In [ ]:
def MINCP_version1_query(args, q):

    center, e, Ds, H_mat, buckets = args[0]
    m = args[1]
    k, L = args[2]
    num_rotations = args[3]

    q1 = q[0].copy()
    
    d = q1.shape[0]
    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []
    
    H1 = (q1 * H_mat).astype(np.float)
    H1[H1 == 0] = np.inf
    H1 = H1.min(axis=1).reshape(L, k)

    for i in range(L):
        q3 = q2.copy()
        for j in range(num_rotations):
            q3 = q3 * Ds[i][j]
            ffht.fht(q3)
        rotate = norm_l2.fit_transform([q3])[0]
        H2 = np.abs(rotate.dot(e.T)).argsort()[::-1][:m]

        for h2 in H2:
            H = np.append(H1[i], h2)
            bi = generate_md5(H)
            if bi in buckets[i]:
                result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 4
L = 13
num_rotations = 3
lsh = MINCP_version1(train, k=k, L=L, num_rotations=num_rotations)

In [ ]:
m = 2

In [ ]:
center, e, Ds, H_mat, buckets = lsh
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, MINCP_version1_query, [lsh, m, (k, L), num_rotations], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## MINCP version 2

### 处理

In [ ]:
def MINCP_version2(P, k, L, num_rotations=3, random_state=None):
    """
    Desc:
    Args:
    Returns:
        
    """
    n = P.shape[0]
    d = P.shape[2]

    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    buckets = []
    for i in range(L):
        buckets.append(dict())
    
    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState(random_state)
    
    num_hash = k * L

    # 1. 生成 hash 函数
    h_mat = np.arange(1, d + 1)
    H_mat = np.array([rand.permutation(h_mat) for _ in range(num_hash)])

    e = generate_cross_polytope(d, random_state) # 生成正轴型向量 2 * d 个
    Ds = rand.choice([-1, 1], (L, num_rotations, d))

    H2 = []
    for i in range(L):
        rotate = []
        for p in P2:
            for j in range(num_rotations):
                p = p * Ds[i][j]
                ffht.fht(p)
            rotate.append(p)
        rotate = norm_l2.fit_transform(np.array(rotate)) # 加速
        H2.append(np.abs(rotate.dot(e[:d].T)).argmax(axis=1))

    for idx, p in enumerate(P1):
        H1 = (p * H_mat).astype(np.float)
        H1[H1 == 0] = np.inf
        H1 = H1.min(axis=1).reshape(L, k)
        for i in range(L):
            H = np.append(H1[i], H2[i][idx])
            bi = generate_md5(H)
            if bi not in buckets[i]:
                buckets[i][bi] = [idx]
            else:
                buckets[i][bi].append(idx)

    return center, e, Ds, H_mat, buckets

### 查询

In [ ]:
def MINCP_version2_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
    """
    center, e, Ds, H_mat, buckets = args[0]
    m = args[1]
    k, L = args[2]
    num_rotations = args[3]

    q1 = q[0].copy()
    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]

    result = []

    d = q1.shape[0]
    
    H1 = (q1 * H_mat).astype(np.float)
    H1[H1 == 0] = np.inf
    H1 = H1.min(axis=1).reshape(L, k)

    for i in range(L):
        q3 = q2.copy()
        for j in range(num_rotations):
            q3 = q3 * Ds[i][j]
            ffht.fht(q3)
        rotate = norm_l2.fit_transform([q3])[0]
        H2 = np.abs(rotate.dot(e[:d].T)).argsort()[::-1][:m]

        for h2 in H2:
            H = np.append(H1[i], h2)
            bi = generate_md5(H)
            if bi in buckets[i]:
                result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k = 4
L = 15
num_rotations = 3
lsh = MINCP_version2(train, k=k, L=L, num_rotations=num_rotations)

In [ ]:
m = 3

In [ ]:
center, e, Ds, H_mat, buckets = lsh
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, MINCP_version2_query, [lsh, m, (k, L), num_rotations], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## MINHASHHP(Min-Hash and Hyperplane)

### 处理

In [ ]:
def MINHP_process(P, k1, k2, L, random_state=None):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        L: 桶数
    Returns:
    """
    n = P.shape[0]
    d = P.shape[2]

    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    buckets = []
    for i in range(L):
        buckets.append(dict())
    
    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState(random_state)
    
    num_hash = k1 * L

    # 1. 生成 hash 函数
    h_mat = np.arange(1, d + 1)
    H_mat = np.array([rand.permutation(h_mat) for _ in range(num_hash)])
    V = np.array([norm_l2.fit_transform(V) for V in rand.normal(0, 1, (L, k2, d))])

    H2 = []
    for i in range(L):
        H2.append(np.array(P2.dot(V[i].T) > 0, dtype=np.int))

    for idx, p in enumerate(P1):
        H1 = (p * H_mat).astype(np.float)
        H1[H1 == 0] = np.inf
        H1 = H1.min(axis=1).reshape(L, k1)
        for i in range(L):
            H = np.append(H1[i], H2[i][idx]).astype(np.int)

            bi = generate_md5(H)
            if bi not in buckets[i]:
                buckets[i][bi] = [idx]
            else:
                buckets[i][bi].append(idx)

    return center, H_mat, V, buckets

### 查询

In [ ]:
def MINHP_query(args, q):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
    """
    center, H_mat, V, buckets = args[0]
    k1, k2, L = args[1]
    
    q1 = q[0].copy()
    # 正则化至单位球面
    q2 = q[1].copy()
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]
    result = []
    
    H1 = (q1 * H_mat).astype(np.float)
    H1[H1 == 0] = np.inf
    H1 = H1.min(axis=1).reshape(L, k1)
    
    for i in range(L):
        H2 = np.array(q2.dot(V[i].T) > 0, dtype=np.int)

        H = np.append(H1[i], H2).astype(np.int)

        bi = generate_md5(H)
        if bi in buckets[i]:
            result.append(buckets[i][bi])

    if len(result) != 0:

        result = np.unique(np.concatenate(result))

    return result

### 测试

In [ ]:
%%time
k1 = 2
k2 = 4
L = 42
lsh = MINHP_process(train, k1, k2, L)

In [ ]:
center, H_mat, V, buckets = lsh

In [ ]:
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, MINHP_query, [lsh, (k1, k2, L)], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## TLOPRCP 双层过滤

In [ ]:
def generate_cross_polytope(d, random_state=None):
    """
    Desc:
        生成正轴型 d 维, 2 * d 个
    """
    if random_state is not None:
        rand = np.random.RandomState(random_state)
    else:
        rand = np.random.RandomState()
    A = rand.normal(0, 1, (d, d))
    A = norm_l2.fit_transform(gram_schmidt(A))
    A = np.append(A, -A).reshape(-1, d)
    return A

### 结构

In [ ]:
def TLOPRCP(P, k, L1, L2, random_state=None):
    """
    Desc:
    Args:
        P: 点集 n * 2 * d
        k: 分段数
            sd: D / k 每段长度
        L: 桶数
        C: 防碰撞参数
        
    Returns:
        
    """
    n = P.shape[0]
    d = P.shape[2]
    sd = int(d / k)
    C = math.ceil(d / k + L1) + 2
    P1 = P[:, 0].copy()
    P2 = P[:, 1].copy()
    center = np.mean(P2, axis=0)
    P2 = P2 - center
    P2 = norm_l2.fit_transform(P2)

    print('k = {}, L = {}'.format(k, L1))
    print('Every segament length is {}'.format(sd))

    seeds = np.random.choice(np.arange(k * L1 * d), L1, replace=False)
    one_segament_vector = list(range(sd, 0, -1))
    auxiliary_vector = np.array(one_segament_vector * k)
    e = generate_cross_polytope(d, random_state) # 生成正轴型向量 2 * d 个
    As = np.array([norm_l2.fit_transform(A) for A in np.random.normal(0, 1, (L2, d, d))]) # 生成辅助旋转矩阵
    idxs = np.arange(n)

    buckets1 = []
    buckets2 = []
    for i in range(L1):
        bucket = dict()
        for idx, p in enumerate(P):
            np.random.seed(seeds[i])
            php = np.random.permutation(p[0])
            H1 = -((auxiliary_vector * php).reshape(-1, sd).max(axis=1) - sd)
            H1[H1 == sd] = -1
            for j, h in enumerate(H1):
                if h == -1:
                    nj = j
                    while H1[nj] == -1:
                        nj = (1 + nj) % k
                    H1[j] = H1[nj] + C
    
            bi = generate_md5(H1)
            if bi not in bucket:
                bucket[bi] = [idx]
            else:
                bucket[bi].append(idx)

        buckets1.append(bucket)
    
    
    idxs = np.arange(n)
    for i in range(L2):
        bucket = dict()
        A = As[i]
        rotate = norm_l2.fit_transform([A.dot(p) for p in P2]) # 加速
        H2 = rotate.dot(e.T).argmax(axis=1)
        
        for i in range(e.shape[0]):
            part = idxs[H2 == i]
            if len(part) != 0:
                bucket[i] = part

        buckets2.append(bucket)
        
    result = dict()
    result['center'] = center
    result['e'] = e
    result['As'] = As
    result['P'] = transform_hybrid(P1, P2, d)
    result['seeds'] = seeds
    result['auxiliary'] = auxiliary_vector
    result['buckets1'] = buckets1
    result['buckets2'] = buckets2
    result['k'] = k
    result['L1'] = L1
    result['L2'] = L2
    
    return result

### 查询

In [ ]:
def TLOPRCP_query(args, q):
    P = args[0]['P']
    n = P.shape[0]
    d = P.shape[2]
    center = args[0]['center']
    k = args[0]['k']
    L1 = args[0]['L1']
    L2 = args[0]['L2']
    seeds = args[0]['seeds']
    auxiliary_vector = args[0]['auxiliary']
    e = args[0]['e']
    sd = int(d / k)
    C = math.ceil(d / k + L1) + 2
    buckets1 = args[0]['buckets1']
    buckets2 = args[0]['buckets2']
    As = args[0]['As']
    
    m = args[1]

    q1 = q[0]
    q2 = q[1]
    q2 = q2 - center
    q2 = norm_l2.fit_transform([q2])[0]
    result1 = []
    result2 = []
    # 计算hamming
    for i in range(L1):
        np.random.seed(seeds[i])
        php = np.random.permutation(q1)
        H1 = -((auxiliary_vector * php).reshape(-1, sd).max(axis=1) - sd)
        H1[H1 == sd] = -1
        for j, h in enumerate(H1):
            if h == -1:
                nj = j
                while H1[nj] == -1:
                    nj = (1 + nj) % k
                H1[j] = H1[nj] + C

        bi = generate_md5(H1)
        if bi in buckets1[i]:
            result1.append(buckets1[i][bi])
    

    for i in range(L2):
        bucket = buckets2[i]
        A = As[i]
        rotate = norm_l2.fit_transform([A.dot(q2)])[0]
        H = rotate.dot(e.T).argsort()[::-1][:m]
        for h in H:
            if h in bucket:
                result2.append(bucket[h])

                                
    if len(result1) != 0 and len(result2) != 0:
        result1 = np.unique(np.concatenate(result1))
        result2 = np.unique(np.concatenate(result2))
        result = np.intersect1d(result1, result2)

    return result

### 测试

In [ ]:
%%time
k = 4
L1 = 10
L2 = 5
m = 3
tloprcp = TLOPRCP(train, k, L1, L2)

In [ ]:
m = 2

In [ ]:
buckets1, buckets2 = tloprcp['buckets1'], tloprcp['buckets2']

In [ ]:
size = 0
for bucket in buckets1:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
for bucket in buckets2:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, TLOPRCP_query, [tloprcp, m], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## TLMinHashE2LSH

### 处理函数

In [ ]:
class TLMinE2LSH(object):
    def __init__(self, P):
        P1 = P[:, 0].copy()
        P2 = P[:, 1].copy()
        self.P = P
        self.hamming_P = P1
        self.eculidean_P = P2

        self.n_ = self.hamming_P.shape[0]
        self.d_ = self.hamming_P.shape[1]

        # 设置 r1
        self.set_r1()
        self.set_d1()
    
    def _logab(self, a, b):
        """
        Desc:
            求解 log_a b
        """
        return np.log(b) / np.log(a)

    def _Pr(self, w, c):
        """
        Args:
            w: 段长
            c: r1, r2, 距离
        """
        a = 2 * norm.cdf(-w/c)
        b = 2 / (np.sqrt(2 * np.pi) * w / c)
        d = np.e ** (-((w**2) / (2 * (c ** 2))))    
        return 1 - a - b * (1 - d)
    
    def _rho(self, p1, p2):
        return (np.log(1/p1) / np.log(1/p2))

    def set_r1(self, r1=None):
        """
        Desc:
            获取数据集 r1
        """
        P = self.hamming_P
        print('Estimate r1')
        print('P: {}'.format(P.shape))
        if r1 is not None:
            self.r1_ = r1
            print('set r1 = {}'.format(r1))
            return

        n = 20
        nn = int(np.log(P.shape[0]))
        print('sample {} points, calculate near {} neighbors.'.format(n, nn))
        idx = np.random.choice(P.shape[0], size=n, replace=False)
        qs = P[idx]
        r1_avg = []
        for q in qs:
            dist_arr = np.array([distance.minkowski(q, p, p=1) for p in P])
            dist_sort = np.sort(dist_arr)
            avg = np.sort(dist_arr)[1:nn+1].mean()
            r1_avg.append(avg)
        r1 = math.ceil(np.array(r1_avg).mean())
        self.r1_ = r1
        print('estimate r1 = {}'.format(self.r1_))
        return r1

    def set_d1(self, d1=None):
        """
        Desc:
            获取数据集 d1
        """
        P = self.eculidean_P
        d = P.shape[1]
        print('Estimate d1')
        print('P: {}'.format(P.shape))
        if d1 is not None:
            self.d1_ = d1
            print('set d1 = {}'.format(d1))
            return

        n = 20
        nn = int(np.log(P.shape[0]))
        print('sample {} points, calculate near {} neighbors.'.format(n, nn))
        idx = np.random.choice(P.shape[0], size=n, replace=False)
        qs = P[idx]
        d1_avg = []
        for q in qs:
            dist_arr = np.array([distance.euclidean(q, p) for p in P])
            dist_sort = np.sort(dist_arr)
            avg = np.sort(dist_arr)[1:nn+1].mean()
            d1_avg.append(avg)
        d1 = np.array(d1_avg).mean()
        
        self.d1_ = d1
        print('estimate d1 = {}'.format(self.d1_))
        return d1

    def draw_W(self, c=2):
        """
        Desc:
            绘制图像，观察最优 w 值
        Args:
            c: 近似因子
        """
        d1 = self.d1_
        W = np.arange(d1 / 2, d1 * 10, d1 / 2)
        self.c_ = c
        self.d2_ = c * d1
    
        p1 = [self._Pr(w, d1) for w in W]
        p2 = [self._Pr(w, c * d1) for w in W]
        rho = [self._rho(p1[i], p2[i]) for i in range(len(W))]

        plt.plot(W, p1, label='p1')
        plt.plot(W, p2, label='p2')
        plt.plot(W, rho, label='rho')
        plt.hlines(y=1/c, xmin=0, xmax=d1 * 10, label='1/c')
        plt.xlabel('w')
        plt.ylabel('p')
        plt.legend()
        plt.show()
        
    def set_k(self, k1, k2):
        self.k1_ = k1
        self.k2_ = k2
    def set_L(self, L1, L2):
        self.L1_ = L1
        self.L2_ = L2

    def set_param(self, w, delta=np.e):
        n = self.hamming_P.shape[0]
        self.r2_ = self.r1_ * self.c_
        self.d2_ = self.d1_ * self.c_
        d = self.d_
        self.delta_ = delta
        self.w_ = w

        # hamming 距离
        p1 = 1 - self.r1_ / d
        p2 = 1 - self.r2_ / d
        rho = self._rho(p1, p2)
        k1 = math.ceil(self._logab(p2, 1/n))
        self.hamming_p1_ = p1
        self.hamming_p2_ = p2
        self.k1_ = k1
        self.hamming_rho_ = rho
        

        euclidean_p1 = self._Pr(w, self.d1_)
        euclidean_p2 = self._Pr(w, self.d2_)
        rho = self._rho(p1, p2)
        k2 = math.ceil(self._logab(p2, 1/n))
        self.euclidean_p1_ = euclidean_p1
        self.euclidean_p2_ = euclidean_p2
        self.k2_ = k2
        self.euclidean_rho_ = rho

        if self.delta_ == np.e:
            L1 = math.ceil(n ** self.hamming_rho_)
            L2 = math.ceil(n ** self.euclidean_rho_)
        else:
            L1 = math.ceil(math.log( 1 / self.delta_) / -math.log(1 - self.hamming_p1_ ** self.k1_))  
            L2 = math.ceil(math.log( 1 / self.delta_) / -math.log(1 - self.euclidean_p1_ ** self.k2_))  
    
        self.L1_ = L1
        self.L2_ = L2
        print('E2LSH set w = {}'.format(w))
        print('For hamming: k = {}, p1 = {}, p2 = {}'.format(k1, self.hamming_p1_, self.hamming_p2_))
        print('For euclidean: k = {}, p1 = {}, p2 = {}'.format(k2, self.euclidean_p1_, self.euclidean_p2_))
        print('L1 = {}, L2 = {}'.format(L1, L2))

    def generate_auxiliary_vector(self):
        """
        Desc:
            生成 G, 生成 H_mat
        """

        d = self.d_
        w = self.w_
        k1 = self.k1_
        k2 = self.k2_
        L1 = self.L1_
        L2 = self.L2_
        print('Generate auxiliary vector')
        # 生成 G
        h_mat = np.arange(1, d + 1)
        H_mat = np.array([np.random.permutation(h_mat) for _ in range(k1 * L1)])
        self.H_mat_ = H_mat
        
        G = []
        for i in range(L2):
            g = []
            for i in range(k2):
                a = np.random.normal(0, 1, d)
                b = np.random.uniform(0, w)
                g.append((a, b))
            G.append(g)
        self.G_ = G

### 处理函数

In [ ]:
def TLMinE2LSH_process(minhash_e2lsh_param):
    P = minhash_e2lsh_param.P
    n = P.shape[0]
    d = minhash_e2lsh_param.hamming_P.shape[1]

    L1 = minhash_e2lsh_param.L1_
    L2 = minhash_e2lsh_param.L2_
    # for hamming
    k1 = minhash_e2lsh_param.k1_
    H_mat = minhash_e2lsh_param.H_mat_

    # for euclidean
    k2 = minhash_e2lsh_param.k2_
    G = minhash_e2lsh_param.G_
    w = minhash_e2lsh_param.w_

    buckets1 = []
    buckets2 = []
    for i in range(L1):
        buckets1.append(dict())
    for i in range(L2):
        buckets2.append(dict())

    for idx, p in enumerate(P):
        hamming_p = p[0]

        h1 = (hamming_p * H_mat).astype(np.float)
        h1[h1 == 0] = np.inf
        h1 = h1.min(axis=1).reshape(L1, k1)
        for i in range(L1):
            bi = generate_md5(h1[i])
            if bi not in buckets1[i]:
                buckets1[i][bi] = [idx]
            else:
                buckets1[i][bi].append(idx)
        
        euclidean_p = p[1]
        for i in range(L2):
            g = G[i]
            g_val = np.array([math.ceil((euclidean_p.dot(h[0]) + h[1]) / w) for h in g])
            bi = generate_md5(g_val)

            if bi not in buckets2[i]:
                buckets2[i][bi] = [idx]
            else:
                buckets2[i][bi].append(idx)

    return buckets1, buckets2

### 查询函数

In [ ]:
def TLMinE2LSH_query(args, q):
    """
    Desc:
    Args:
        P: 点集
        args:
            [0]: hybird_E2LSH_param
            [1]: [buckets1, buckets2]
        q: 查询点
    """
    P_ = args[0].P
    n = args[0].n_
    d = args[0].d_

    L1 = args[0].L1_
    L2 = args[0].L2_

    # for hamming
    k1 = args[0].k1_
    H_mat = args[0].H_mat_
    # for euclidean
    k2 = args[0].k2_
    w = args[0].w_

    G = args[0].G_

    buckets1 = args[1][0]
    buckets2 = args[1][1]

    q1 = q[0]
    q2 = q[1]

    result1 = []
    result2 = []

    h = (q1 * H_mat).astype(np.float)
    h[h == 0] = np.inf
    h = h.min(axis=1).reshape(L1, k1)
    for i in range(L1):
        bi = generate_md5(h[i])
        if bi in buckets1[i]:
            result1.append(buckets1[i][bi])

    for i in range(L2):
        g = G[i]
        g_val = np.array([math.ceil((q2.dot(h[0]) + h[1]) / w) for h in g])
        bi = generate_md5(g_val)
        
        if bi in buckets2[i]:
            result2.append(buckets2[i][bi])

    result = []
    if len(result1) != 0 and len(result2) != 0:
        result1 = np.unique(np.concatenate(result1))
        result2 = np.unique(np.concatenate(result2))
        result = np.intersect1d(result1, result2)
    
    return result

### 测试

In [ ]:
tlminhashe2lsh = TLMinE2LSH(P=train)
tlminhashe2lsh.draw_W(c=3)

In [ ]:
tlminhashe2lsh.set_param(200, delta=0.1)

In [ ]:
tlminhashe2lsh.set_k(8, 10)

In [ ]:
tlminhashe2lsh.set_L(150, 400)

In [ ]:
tlminhashe2lsh.set_L(100, 400)
tlminhashe2lsh.generate_auxiliary_vector()

In [ ]:
tlminhashe2lsh.generate_auxiliary_vector()

In [ ]:
%%time
buckets = TLMinE2LSH_process(tlminhashe2lsh)

In [ ]:
size = 0
for bucket in buckets:
    for bu in bucket:
        for key, b in bu.items():
            size += sys.getsizeof(key)
            size += sys.getsizeof(b)
size / mbyte

In [ ]:
%%time
metrics(train, TLMinE2LSH_query, [tlminhashe2lsh, buckets], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## Hybrid E2LSH

### 处理类

In [ ]:
class HybridE2LSH(object):
    def __init__(self, P):
        self.P = P.copy()
        self.P = self.P.astype(np.float)
        self.P[:, 1] = np.array(norm_l2.fit_transform(self.P[:, 1]), dtype=np.float)
        # print(self.P[0][1])
        self.hamming_P = self.P[:, 0]
        self.eculidean_P = self.P[:, 1]

        self.n_ = self.hamming_P.shape[0]
        self.d_ = self.hamming_P.shape[1]

        # 设置 r1
        self.set_r1(self.d_)
        self.set_d1()

    def _logab(self, a, b):
        """
        Desc:
            求解 log_a b
        """
        return np.log(b) / np.log(a)

    def _Pr(self, w, c, nm):
        """
        Args:
            w: 段长
            c: r1, r2, 距离
        """ 
        if nm == 'l2':
            a = 2 * norm.cdf(-w/c)
            b = 2 / (np.sqrt(2 * np.pi) * w / c)
            d = np.e ** (-((w**2) / (2 * (c ** 2))))    
            return 1 - a - b * (1 - d)
        elif nm == 'l1':
            a = w/c
            b = 2 * (np.arctan(a)) / np.pi
            d = 1 / (np.pi * a)
            e = np.log(1 + (a**2))
            return b - d * e
    
    def _rho(self, p1, p2):
        return (np.log(1/p1) / np.log(1/p2))

    def set_r1(self, r1=None):
        """
        Desc:
            获取数据集 r1
        """
        P = self.hamming_P
        print('Estimate r1')
        print('P: {}'.format(P.shape))
        if r1 is not None:
            self.r1_ = r1
            print('set r1 = {}'.format(r1))
            return

        n = 20
        nn = int(np.log(P.shape[0]))
        print('sample {} points, calculate near {} neighbors.'.format(n, nn))
        idx = np.random.choice(P.shape[0], size=n, replace=False)
        qs = P[idx]
        r1_avg = []
        for q in qs:
            dist_arr = np.array([distance.minkowski(q, p, p=1) for p in P])
            dist_sort = np.sort(dist_arr)
            avg = np.sort(dist_arr)[1:nn+1].mean()
            r1_avg.append(avg)
        r1 = math.ceil(np.array(r1_avg).mean())
        self.r1_ = r1
        print('estimate r1 = {}'.format(self.r1_))
        return r1

    def set_d1(self, d1=None):
        """
        Desc:
            获取数据集 d1
        """
        P = self.eculidean_P
        d = P.shape[1]
        print('Estimate d1')
        print('P: {}'.format(P.shape))
        if d1 is not None:
            self.d1_ = d1
            print('set d1 = {}'.format(d1))
            return

        n = 20
        nn = int(np.log(P.shape[0]))
        print('sample {} points, calculate near {} neighbors.'.format(n, nn))
        idx = np.random.choice(P.shape[0], size=n, replace=False)
        qs = P[idx]
        d1_avg = []
        for q in qs:
            dist_arr = np.array([distance.euclidean(q, p) for p in P])
            dist_sort = np.sort(dist_arr)
            avg = np.sort(dist_arr)[1:nn+1].mean()
            d1_avg.append(avg)
        d1 = np.array(d1_avg).mean()
        
        self.d1_ = d1
        print('estimate d1 = {}'.format(self.d1_))
        return d1

    def draw_W(self, c1=4, c2=2):
        """
        Desc:
            绘制图像，观察最优 w 值
        Args:
            c: 近似因子
        """
        self.c1_ = c1
        plt.figure(figsize=(10, 5))
        
        r1 = self.r1_
        Wr = np.arange(r1 / 2, r1 * 10, r1)
        self.r2_ = c1 * r1
        
        d1 = self.d1_
        Wd = np.arange(d1 / 2, d1 * 10, d1 / 2)
        self.d2_ = c2 * d1

        plt.subplot(121)
        plt.title('hamming')
        p1 = [self._Pr(w, r1, 'l1') for w in Wr]
        p2 = [self._Pr(w, c1 * r1, 'l1') for w in Wr]
        rho = [self._rho(p1[i], p2[i]) for i in range(len(Wr))]

        plt.plot(Wr, p1, label='p1')
        plt.plot(Wr, p2, label='p2')
        plt.plot(Wr, rho, label='rho')
        plt.hlines(y=1/c1, xmin=0, xmax=r1 * 10, label='1/c')
        plt.xlabel('w')
        plt.ylabel('p')
        plt.legend()
        
        
        
        plt.subplot(122)
        plt.title('euclidean')
        p1 = [self._Pr(w, d1, 'l2') for w in Wd]
        p2 = [self._Pr(w, c2 * d1, 'l2') for w in Wd]
        rho = [self._rho(p1[i], p2[i]) for i in range(len(Wd))]

        plt.plot(Wd, p1, label='p1')
        plt.plot(Wd, p2, label='p2')
        plt.plot(Wd, rho, label='rho')
        plt.hlines(y=1/c2, xmin=0, xmax=d1 * 10, label='1/c')
        plt.xlabel('w')
        plt.ylabel('p')
        plt.legend()

        plt.show()

    def set_k1_k2(self, k1, k2):
        self.k1_ = k1
        self.k2_ = k2

    def set_L(self, L):
        self.L_ = L

    def set_param(self, w1, w2, delta=np.e):
        self.delta_ = delta
        
        n = self.n_

        # hamming 距离
        p1 = self._Pr(w1, self.r1_, 'l1')
        p2 = self._Pr(w1, self.r2_, 'l1')
        rho = self._rho(p1, p2)
        k1 = math.ceil(self._logab(p2, (1 - np.sqrt(1 - 1 / n))))
        
        print('For Hamming')
        print('w1 = {}'.format(w1))
        print('p1 = {:.4f}, p2 = {:.4f}, rho = {:.4f}'.format(p1, p2, rho))
        print('k1 = {}'.format(k1))
        
        self.hamming_p1_ = p1
        self.hamming_p2_ = p2
        self.hamming_rho_ = rho
        

        # 欧式距离
        p1 = self._Pr(w2, self.d1_, 'l2')
        p2 = self._Pr(w2, self.d2_, 'l2')
        rho = self._rho(p1, p2)
        k2 = math.ceil(self._logab(p2, (1 - np.sqrt(1 - 1/ n))))

        print('For Euclidean')
        print('w2 = {}'.format(w2))
        print('p1 = {:.4f}, p2 = {:.4f}, rho = {:.4f}'.format(p1, p2, rho))
        print('k2 = {}'.format(k2))
        
        self.euclidean_p1_ = p1
        self.euclidean_p2_ = p2
        self.euclidean_rho_ = rho
        
        self.k1_ = k1
        self.w1_ = w1
        self.k2_ = k2
        self.w2_ = w2

                    
        if self.delta_ == np.e:
            L = math.ceil((1 - np.sqrt(1 - 1/n)) ** (-(
                self.hamming_rho_ + self.euclidean_rho_)))
        else:
            L = math.ceil(math.log( 1 / self.delta_) /
                          -math.log(1 - (1 - (np.sqrt(1-1/n))) ** (self.hamming_rho_ + self.euclidean_rho_)))
        
        self.L_ = L
        print('L = {}'.format(L))
    
    def generate_auxiliary_vector(self):
        """
        Desc:
            生成 G, 生成 H_mat
        """

        d = self.d_
        w1 = self.w1_
        w2 = self.w2_
        k1 = self.k1_
        k2 = self.k2_
        L = self.L_
        print('Generate auxiliary vector: G.')
        print('For each g in G (total {}), have {} hash functions.'.format(L, k2))

        # 生成 G
        G1 = []
        G2 = []

        for i in range(L):
            g = []
            for i in range(k1):
                a = np.random.standard_cauchy(d)
                b = np.random.uniform(0, w1)
                g.append((a, b))
            G1.append(np.array(g))

            g = []
            for i in range(k2):
                a = np.random.normal(0, 1, d)
                b = np.random.uniform(0, w2)
                g.append((a, b))
            G2.append(np.array(g))
            
        self.fp_rands_ = np.random.randint(1, 6 * (k1 + k2), (L, k1 + k2))
            
        self.G1_ = np.array(G1)
        self.G2_ = np.array(G2)
        

### 处理函数

In [ ]:
def HybridE2LSH_process(param):
    P = param.P
    n = P.shape[0]
    d = param.hamming_P.shape[1]

    L = param.L_
    # for hamming
    k1 = param.k1_
    w1 = param.w1_
    
    # for euclidean
    k2 = param.k2_
    w2 = param.w2_
    
    G1 = param.G1_
    G2 = param.G2_
    fp_rands = param.fp_rands_
    buckets = []
    C = 2 ** 32 - 5

    for i in range(L):
        buckets.append(dict())

    for idx, p in enumerate(P):
        hamming_p = p[0]
        euclidean_p = p[1]

        for i in range(L):
            g1 = G1[i]
            g1_val = np.array([math.ceil((hamming_p.dot(h[0]) + h[1]) / w1) for h in g1])

            g2 = G2[i]
            g2_val = np.array([math.ceil((euclidean_p.dot(h[0]) + h[1]) / w2) for h in g2])

            h = np.append(g1_val, g2_val)

            
            bi = generate_md5(h)
            
            if bi not in buckets[i]:
                buckets[i][bi] = [idx]
            else:
                buckets[i][bi].append(idx)

    return buckets

### 查询函数

In [ ]:
def HybridE2LSH_query(args, q):
    """
    Desc:
    Args:
        P: 点集
        args:
            [0]: hybird_E2LSH_param
            [1]: buckets
        q: 查询点
    """
    P = args[0].P
    n = args[0].n_
    d = args[0].d_


    L = args[0].L_

    # for hamming
    k1 = args[0].k1_
    w1 = args[0].w1_

    # for euclidean
    k2 = args[0].k2_
    w2 = args[0].w2_

    G1 = args[0].G1_
    G2 = args[0].G2_

    buckets = args[1]

    q1 = q[0]
    q2 = q[1]

    q2 = norm_l2.fit_transform([q2])[0]
    
    # print(q1)
    # print(q2)

    result = []
    
    for i in range(L):
        g1 = G1[i]
        g1_val = np.array([math.ceil((q1.dot(h[0]) + h[1]) / w1) for h in g1])
    
        g2 = G2[i]
        g2_val = np.array([math.ceil((q2.dot(h[0]) + h[1]) / w2) for h in g2])
        
        h = np.append(g1_val, g2_val)
        
            
        bi = generate_md5(h)

        if bi in buckets[i]:
            result.append(buckets[i][bi])

    if len(result) != 0:
        result = np.unique(np.concatenate(result))
    
    return result

### 测试

In [ ]:
%%time
lsh = HybridE2LSH(train)
lsh.draw_W(c1=3,c2=3)

In [ ]:
lsh.set_param(w1=500, w2=3, delta=0.1)

In [ ]:
lsh.set_k1_k2(7, 7)

In [ ]:
lsh.set_L(500)
lsh.generate_auxiliary_vector()

In [ ]:
%%time
buckets = HybridE2LSH_process(lsh)

In [ ]:
size = 0
for bucket in buckets:
    for key, value in bucket.items():
        size += sys.getsizeof(key)
        size += sys.getsizeof(value)
size / mbyte

In [ ]:
%%time
metrics(train, HybridE2LSH_query, [lsh, buckets], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)

## Two Level E2LSH

#### 参数类

In [ ]:
class TLE2LSH(object):
    def __init__(self, P):
        P1 = P[:, 0].copy()
        P2 = P[:, 1].copy()
        d = P2.shape[1]
        self.P = P
        self.hamming_P = P1
        self.eculidean_P = P2

        self.n_ = self.hamming_P.shape[0]
        self.d_ = self.hamming_P.shape[1]

        # 设置 r1
        self.set_r1()
        self.set_d1()

    def _logab(self, a, b):
        """
        Desc:
            求解 log_a b
        """
        return np.log(b) / np.log(a)

    def _Pr(self, w, c, nm):
        """
        Args:
            w: 段长
            c: r1, r2, 距离
        """ 
        if nm == 'l2':
            a = 2 * norm.cdf(-w/c)
            b = 2 / (np.sqrt(2 * np.pi) * w / c)
            d = np.e ** (-((w**2) / (2 * (c ** 2))))    
            return 1 - a - b * (1 - d)
        elif nm == 'l1':
            a = w/c
            b = 2 * (np.arctan(a)) / np.pi
            d = 1 / (np.pi * a)
            e = np.log(1 + (a**2))
            return b - d * e
    
    def _rho(self, p1, p2):
        return (np.log(1/p1) / np.log(1/p2))

    def set_r1(self, r1=None):
        """
        Desc:
            获取数据集 r1
        """
        P = self.hamming_P
        print('Estimate r1')
        print('P: {}'.format(P.shape))
        if r1 is not None:
            self.r1_ = r1
            print('set r1 = {}'.format(r1))
            return

        n = 20
        nn = int(np.log(P.shape[0]))
        print('sample {} points, calculate near {} neighbors.'.format(n, nn))
        idx = np.random.choice(P.shape[0], size=n, replace=False)
        qs = P[idx]
        r1_avg = []
        for q in qs:
            dist_arr = np.array([distance.minkowski(q, p, p=1) for p in P])
            dist_sort = np.sort(dist_arr)
            avg = np.sort(dist_arr)[1:nn+1].mean()
            r1_avg.append(avg)
        r1 = math.ceil(np.array(r1_avg).mean())
        self.r1_ = r1
        print('estimate r1 = {}'.format(self.r1_))
        return r1

    def set_d1(self, d1=None):
        """
        Desc:
            获取数据集 d1
        """
        P = self.eculidean_P
        d = P.shape[1]
        print('Estimate d1')
        print('P: {}'.format(P.shape))
        if d1 is not None:
            self.d1_ = d1
            print('set d1 = {}'.format(d1))
            return

        n = 20
        nn = int(np.log(P.shape[0]))
        print('sample {} points, calculate near {} neighbors.'.format(n, nn))
        idx = np.random.choice(P.shape[0], size=n, replace=False)
        qs = P[idx]
        d1_avg = []
        for q in qs:
            dist_arr = np.array([distance.euclidean(q, p) for p in P])
            dist_sort = np.sort(dist_arr)
            avg = np.sort(dist_arr)[1:nn+1].mean()
            d1_avg.append(avg)
        d1 = np.array(d1_avg).mean()
        
        self.d1_ = d1
        print('estimate d1 = {}'.format(self.d1_))
        return d1

    def draw_W(self, c1=3, c2=3):
        """
        Desc:
            绘制图像，观察最优 w 值
        Args:
            c: 近似因子
        """
        self.c1_ = c1
        plt.figure(figsize=(10, 5))

        r1 = self.r1_
        Wr = np.arange(r1 / 2, r1 * 10, r1)
        self.r2_ = c1 * r1
        
        d1 = self.d1_
        Wd = np.arange(d1 / 2, d1 * 10, d1 / 2)
        self.d2_ = c2 * d1

        plt.subplot(121)
        plt.title('hamming')
        p1 = [self._Pr(w, r1, 'l1') for w in Wr]
        p2 = [self._Pr(w, c1 * r1, 'l1') for w in Wr]
        rho = [self._rho(p1[i], p2[i]) for i in range(len(Wr))]

        plt.plot(Wr, p1, label='p1')
        plt.plot(Wr, p2, label='p2')
        plt.plot(Wr, rho, label='rho')
        plt.hlines(y=1/c1, xmin=0, xmax=r1 * 10, label='1/c')
        plt.xlabel('w')
        plt.ylabel('p')
        plt.legend()
        
        plt.subplot(122)
        plt.title('euclidean')
        p1 = [self._Pr(w, d1, 'l2') for w in Wd]
        p2 = [self._Pr(w, c2 * d1, 'l2') for w in Wd]
        rho = [self._rho(p1[i], p2[i]) for i in range(len(Wd))]

        plt.plot(Wd, p1, label='p1')
        plt.plot(Wd, p2, label='p2')
        plt.plot(Wd, rho, label='rho')
        plt.hlines(y=1/c2, xmin=0, xmax=d1 * 10, label='1/c')
        plt.xlabel('w')
        plt.ylabel('p')
        plt.legend()

        plt.show()

    def set_k1_k2(self, k1, k2):
        self.k1_ = k1
        self.k2_ = k2

    def set_L(self, L):
        self.L_ = L

    def set_param(self, w1, w2, delta=np.e):
        self.delta_ = delta

        n = self.n_

        # hamming 距离
        p1 = self._Pr(w1, self.r1_, 'l1')
        p2 = self._Pr(w1, self.r2_, 'l1')
        rho = self._rho(p1, p2)
        k1 = math.ceil(self._logab(p2, 1/n))
        # k1 = math.ceil(self._logab(p2, (1 - np.sqrt(1 - 1 / n))))

        print('For Hamming')
        print('w1 = {}'.format(w1))
        print('p1 = {:.4f}, p2 = {:.4f}, rho = {:.4f}'.format(p1, p2, rho))
        print('k1 = {}'.format(k1))
        
        self.hamming_p1_ = p1
        self.hamming_p2_ = p2
        self.hamming_rho_ = rho
        

        # 欧式距离
        p1 = self._Pr(w2, self.d1_, 'l2')
        p2 = self._Pr(w2, self.d2_, 'l2')
        rho = self._rho(p1, p2)
        k2 = math.ceil(self._logab(p2, 1/n))
        # k2 = math.ceil(self._logab(p2, (1 - np.sqrt(1 - 1/ n))))

        print('For Euclidean')
        print('w2 = {}'.format(w2))
        print('p1 = {:.4f}, p2 = {:.4f}, rho = {:.4f}'.format(p1, p2, rho))
        print('k2 = {}'.format(k2))
        
        self.euclidean_p1_ = p1
        self.euclidean_p2_ = p2
        self.euclidean_rho_ = rho
        
        self.k1_ = k1
        self.w1_ = w1
        self.k2_ = k2
        self.w2_ = w2

        if self.delta_ == np.e:
            L1 = math.ceil(n ** self.hamming_rho_)
            L2 = math.ceil(n ** self.euclidean_rho_)
        else:
            L1 = math.ceil(math.log( 1 / self.delta_) / -math.log(1 - self.hamming_p1_ ** self.k1_))  
            L2 = math.ceil(math.log( 1 / self.delta_) / -math.log(1 - self.euclidean_p1_ ** self.k2_))  
    
        self.L1_ = L1
        self.L2_ = L2
        print('L1 = {}, L2 = {}'.format(L1, L2))
    
    def generate_auxiliary_vector(self):
        """
        Desc:
            生成 G, 生成 H_mat
        """

        d = self.d_
        w1 = self.w1_
        w2 = self.w2_
        k1 = self.k1_
        k2 = self.k2_
        L1 = self.L1_
        L2 = self.L2_
        # print('Generate auxiliary vector: G.')
        # print('For each g in G (total {}), have {} hash functions.'.format(L, k2))

        # 生成 G
        G1 = []
        G2 = []

        for i in range(L1):
            g = []
            for i in range(k1):
                a = np.random.standard_cauchy(d)
                b = np.random.uniform(0, w1)
                g.append((a, b))
            G1.append(np.array(g))

        for i in range(L2):
            g = []
            for i in range(k2):
                a = np.random.normal(0, 1, d)
                b = np.random.uniform(0, w2)
                g.append((a, b))
            G2.append(np.array(g))
                
        self.G1_ = np.array(G1)
        self.G2_ = np.array(G2)
        

#### 处理函数

In [ ]:
def TLE2LSH_process(param):
    P = param.P
    n = P.shape[0]
    d = param.hamming_P.shape[1]

    # for hamming
    L1 = param.L1_
    k1 = param.k1_
    w1 = param.w1_
    
    # for euclidean
    L2 = param.L2_
    k2 = param.k2_
    w2 = param.w2_
    
    G1 = param.G1_
    G2 = param.G2_

    buckets1 = []
    buckets2 = []

    for i in range(L1):
        buckets1.append(dict())

    for i in range(L2):
        buckets2.append(dict())

    for idx, p in enumerate(P):
        hamming_p = p[0]
        for i in range(L1):
            g = G1[i]
            g_val = np.array([math.ceil((hamming_p.dot(h[0]) + h[1]) / w1) for h in g])
            bi = generate_md5(g_val)
            if bi not in buckets1[i]:
                buckets1[i][bi] = [idx]
            else:
                buckets1[i][bi].append(idx)

        euclidean_p = p[1]
        for i in range(L2):
            g = G2[i]
            g_val = np.array([math.ceil((euclidean_p.dot(h[0]) + h[1]) / w2) for h in g])
            bi = generate_md5(g_val)
            if bi not in buckets2[i]:
                buckets2[i][bi] = [idx]
            else:
                buckets2[i][bi].append(idx)


    return [buckets1, buckets2]

#### 查询函数

In [ ]:
def TLE2LSH_query(args, q):
    """
    Desc:
    Args:
        P: 点集
        args:
            [0]: hybird_E2LSH_param
            [1]: [buckets1, buckets2]
        q: 查询点
    """
    P_ = args[0].P
    n = args[0].n_
    d = args[0].d_

    L1 = args[0].L1_
    L2 = args[0].L2_

    # for hamming
    k1 = args[0].k1_
    w1 = args[0].w1_

    # for euclidean
    k2 = args[0].k2_
    w2 = args[0].w2_

    G1 = args[0].G1_
    G2 = args[0].G2_

    buckets1 = args[1][0]
    buckets2 = args[1][1]

    q1 = q[0]
    q2 = q[1]


    result1 = []
    result2 = []

    for i in range(L1):
        g = G1[i]
        g_val = np.array([math.ceil((q1.dot(h[0]) + h[1]) / w1) for h in g])
        bi = generate_md5(g_val)

        if bi in buckets1[i]:
            result1.append(buckets1[i][bi])

    for i in range(L2):
        g = G2[i]
        g_val = np.array([math.ceil((q2.dot(h[0]) + h[1]) / w2) for h in g])
        bi = generate_md5(g_val)
        
        if bi in buckets2[i]:
            result2.append(buckets2[i][bi])

    if len(result1) == 0:
        print('hamming is empty')
    
    if len(result2) == 0:
        print('euclidean is empty')
        
    result = []
    if len(result1) != 0 and len(result2) != 0:
        result1 = np.unique(np.concatenate(result1))
        result2 = np.unique(np.concatenate(result2))
        result = np.unique(np.concatenate([result1, result2]))
    
    return result

### 测试

In [ ]:
tle2lsh_param = TLE2LSH(P=train)

In [ ]:
tle2lsh_param.draw_W(c1=3,c2=3)

In [ ]:
tle2lsh_param.set_param(100, 200, delta=0.1)
tle2lsh_param.generate_auxiliary_vector()

In [ ]:
%%time
buckets = TLE2LSH_process(tle2lsh_param)

In [ ]:
size = 0
for bucket in buckets:
    for bu in bucket:
        for key, b in bu.items():
            size += sys.getsizeof(key)
            size += sys.getsizeof(b)
size / mbyte

In [ ]:
print(metrics.__doc__)

In [ ]:
%%time
metrics(train, TLE2LSH_query, [tle2lsh_param, buckets], 
        test[:100], dataset_title,
        dic_nearest_neighbor[dataset_title], 
        dic_k_near_neighbors[dataset_title], 
        verbose=True)